# IMPORTAR LIBRERIAS

In [1]:
%pip install pandas
%pip install keras_applications
%pip install tensorflow keras
%pip install ultralytics opencv-python matplotlib
%pip install matplotlib
%pip install ultralytics
%pip install transformers torch
%pip install datasets
%pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import cv2
import torch
import gc
import matplotlib.pyplot as plt
import shutil
#import ultralytics

from PIL import Image
from ultralytics import YOLO
from transformers import AutoModelForObjectDetection, AutoImageProcessor
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

2025-05-16 12:21:08.664022: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 12:21:08.674407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747390868.683716 3710405 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747390868.686518 3710405 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747390868.694095 3710405 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# BOUNDING BOX DE LAS IMAGENES

In [ ]:
# === CONFIGURACIÓN ===
modelo = YOLO("best.pt")  
ruta_imagenes = 'test/images'  
ruta_guardado = 'recortes'  

# Listar imágenes válidas
imagenes = sorted([
    f for f in os.listdir(ruta_imagenes)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

if not imagenes:
    print("⚠️ No se encontraron imágenes en la carpeta.")
    exit()

# Procesar cada imagen
for nombre_img in imagenes:
    ruta_completa = os.path.join(ruta_imagenes, nombre_img)
    imagen = cv2.imread(ruta_completa)

    resultados = modelo(imagen)
    boxes = resultados[0].boxes.xyxy.cpu().numpy() if resultados[0].boxes.xyxy is not None else []

    base_nombre = os.path.splitext(nombre_img)[0]

    if len(boxes) > 0:
        print(f"🟢 {nombre_img}: {len(boxes)} objetos detectados.")
        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)
            recorte = imagen[y1:y2, x1:x2]
            nombre_recorte = f"{base_nombre}_obj{i+1}.jpg"
            cv2.imwrite(os.path.join(ruta_guardado, nombre_recorte), recorte)
            print(f"   → Recorte guardado: {nombre_recorte}")
    else:
        print(f"🔴 {nombre_img}: No hay predicciones. Se guarda la imagen completa.")
        nombre_salida = f"{base_nombre}_completa.jpg"
        cv2.imwrite(os.path.join(ruta_guardado, nombre_salida), imagen)

    del resultados
    torch.cuda.empty_cache()
    gc.collect()


0: 608x640 11 plantas, 3.0ms
Speed: 1.5ms preprocess, 3.0ms inference, 1.4ms postprocess per image at shape (1, 3, 608, 640)
🟢 2024-CEV3-20240602.jpg: 11 objetos detectados.
   → Recorte guardado: 2024-CEV3-20240602_obj1.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj2.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj3.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj4.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj5.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj6.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj7.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj8.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj9.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj10.jpg
   → Recorte guardado: 2024-CEV3-20240602_obj11.jpg

0: 640x640 14 plantas, 3.2ms
Speed: 1.7ms preprocess, 3.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
🟢 CBN-PdlC-A1-20130807.jpg: 14 objetos detectados.
   → Recorte guardado: CBN-PdlC-A1-20130807_obj1.jpg
   → Recorte guardado: CBN-PdlC-

# CLASIFICADOR

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import re
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# === CONFIGURACIÓN ===
modelo_path = 'modelos/clasificadorV4.h5'
ruta_recortes = 'recortes/'
tamaño_entrada = (229, 229)
output_csv = 'predicciones.csv'
csv_plantas = 'plantasMod.csv'

# === CARGAR CSV DE PLANTAS PARA MAPEO DE NOMBRE → ID ===

def normalizar(texto):
    texto = texto.replace('_', ' ')
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode()
    texto = texto.lower()
    texto = re.sub(r'[^a-z ]+', '', texto)  # Solo letras y espacios
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

df_plantas = pd.read_csv(csv_plantas, sep=';')
mapeo_nombre_a_id = df_plantas.drop_duplicates(subset='species')[['species', 'species_id']].set_index('species')['species_id'].to_dict()

mapeo_nombre_a_id = {
    normalizar(nombre): species_id
    for nombre, species_id in zip(df_plantas['species'], df_plantas['species_id'])
}

# === CARGAR MODELO ENTRENADO Y NOMBRES DE CLASES ===
modelo_clasificacion = tf.keras.models.load_model(modelo_path)
clases = sorted(os.listdir('dividido/train'))  

# === CLASIFICAR CADA RECORTE ===
resultados = {}

for archivo in sorted(os.listdir(ruta_recortes)):
    if not archivo.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    # Obtener nombre base de la imagen original
    nombre_base = '_'.join(archivo.split('_')[:-1]) + '.jpg'

    # Procesar imagen
    img_path = os.path.join(ruta_recortes, archivo)
    imagen = load_img(img_path, target_size=tamaño_entrada)
    img_array = img_to_array(imagen) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predicción
    pred = modelo_clasificacion.predict(img_array, verbose=0)
    especie_predicha = clases[np.argmax(pred)] 

    especie_predicha_formateada = normalizar(especie_predicha)
    species_id = mapeo_nombre_a_id.get(especie_predicha_formateada)


    # Convertir a species_id usando el mapeo
    species_id = mapeo_nombre_a_id.get(especie_predicha_formateada)
    if species_id is None:
        print(f"⚠️ No se encontró ID para especie: {especie_predicha}")
        continue

    if nombre_base not in resultados:
        resultados[nombre_base] = set()
    resultados[nombre_base].add(int(species_id))

# === GUARDAR RESULTADOS EN CSV FINAL ===
datos_csv = []
for imagen, ids in resultados.items():
    fila = {
        'quadrat_id': imagen.replace('.jpg', ''),
        'species_ids': str(sorted(list(ids))) 
    }
    datos_csv.append(fila)

df = pd.DataFrame(datos_csv)
df.to_csv(output_csv, index=False)
print(f"✅ Predicciones guardadas en: {output_csv}")


2025-05-19 10:04:50.080917: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 10:04:50.172521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747641890.206094  810908 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747641890.215886  810908 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747641890.288003  810908 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

✅ Predicciones guardadas en: predicciones.csv


# RESULTADOS 

In [4]:
df = pd.read_csv('predicciones.csv')

print(df.head())

             quadrat_id                                        species_ids
0    2024-CEV3-20240602  [1356290, 1357021, 1359673, 1361351, 1362325, ...
1  CBN-PdlC-A1-20130807  [1356596, 1358604, 1359766, 1360928, 1363822, ...
2  CBN-PdlC-A1-20130903  [1358708, 1359208, 1359766, 1361115, 1361351, ...
3  CBN-PdlC-A1-20140721  [1356168, 1358604, 1359596, 1360999, 1361329, ...
4  CBN-PdlC-A1-20140811  [1356448, 1359835, 1361323, 1364150, 1391814, ...


In [ ]:
print(list(mapeo_nombre_a_id.keys())[:10])
